In [ ]:
from __future__ import print_function, division
import re
import pandas as pd
import requests
from bs4 import BeautifulSoup
from urllib.request import urlopen
import urllib
from PIL import Image
import time 


In [ ]:
months = ['january', 'february', 'march', 'april', 'may', 'june', 'july', 'august', 'september', 'october', 'december']
years = ['2014', '2015', '2016']
base_url = 'http://hightimes.com/business/pot-prices-'
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}


def generate_urls(months,years):
    urls = []
    for year in years:
        for month in months:
            url = base_url + month + '-' + year + '-thmq'
            print(url)
            urls.append(url)
    return urls 

def bit_ocr(filename):
    #load image from file
    #uses open OCR API at Bit OCR -- API key needed 
    with open(filename, 'rb') as file:
        headers = {'content-type': 'multipart/form-data'}

        payload = {'apikey': 'xxxxxxxxxxxxxxxx', 'lang': 'eng'}
        files = {'file': file}
        r = requests.post("http://www.bitocr.com/api", data=payload, files=files)
        result = r.json()
        if result['error'] == 0:
            #success
    #         print(result['result'])
            raw = result['result'].split('\n')
            prices = [cell.split(' ')[-1] for cell in raw]
            strains = [cell.split(' ')[:-1] for cell in raw]
            strains = [' '.join(strain) for strain in strains]
            price_index_df = pd.DataFrame({'strain': strains, 'price': prices})
            price_index_df['file'] = filename
            print('Success!')
        else:
            #failed
            print("Error #" + str(result['error_code']) + " " + result['error_message'])

    return price_index_df

        
def index_getter(months, years):
    
    index_urls = []
    
    cropped_fnames = []
    
    strain_prices = []
    
    urls = generate_urls(months, years)
    
    for url in urls: 
        response = requests.get(url,headers=headers)
        page = response.text
        try:
            print('Souping: ' + url)
            soup = BeautifulSoup(page, 'lxml')
            if len(soup.find(class_="entry-content").find_all('img')) > 1:
                index_url = soup.find(class_="entry-content").find_all('img')[1]['src']
            else: 
                index_url = soup.find(class_="entry-content").find('img')['src']
            index_urls.append(index_url)
            print(index_url)
        except: 
            print('404!')
            continue 
            
    for each in index_urls:
        try: 
            filename=each.split('/')[-1]
            print('Getting %s' % filename)
            urllib.request.urlretrieve(each, filename)
        except:
            print('Unable to retrieve %s' % filename)
            pass
         
        try:
            with Image.open(filename) as im:
                strain_and_price_box = (340, 80, im.size[0], im.size[1])
                strainNprice = im.crop(strain_and_price_box)
                cropped_fname = 'cropped_' + filename
                strainNprice.save(cropped_fname)
                cropped_fnames.append(cropped_fname)
        except:
            print('Unable to crop %s' % filename)
            pass
        
    for each in cropped_fnames:
        
        try:
            print("OCRing %s" % cropped_fname)
            bit_ocr(cropped_fname)
            strain_prices.append(price_index_df)
            
#             for point in price_index:
#                 if point[0] not in strain_prices:
#                     strain_prices[point[0]] = point[1]
#                 else:
#                     strain_prices[point[0]].append(point[1])
        except Exception as e: 
            print('Unable to gather data for %s' % cropped_fname)
            print(e)
            pass
        
        print("Pausing for OCR...")
        time.sleep(300)
        
    return strain_prices

In [ ]:
strain_prices = index_getter(months, years)

In [ ]:
price_df = pd.concat(strain_prices).reset_index(drop=True)

In [ ]:
# Bit OCR generates some predictable errors in the form of extraneous leading characters
# This strips the leading characters to clean the strain names up 
price_df['strain'] = price_df['strain'].map(lambda x: x.lstrip('1 '))
price_df['strain'] = price_df['strain'].map(lambda x: x.lstrip('L '))
price_df['strain'] = price_df['strain'].map(lambda x: x.lstrip('V '))
price_df['strain'] = price_df['strain'].map(lambda x: x.lstrip('{ '))
price_df['strain'] = price_df['strain'].map(lambda x: x.lstrip('3 '))